In [14]:
import pandas as pd
import numpy as np

In [2]:
df_labels = pd.read_csv('d_labitems.csv')
df_patients = pd.read_csv('../event_level_cleaned.csv')

In [3]:
df_sample = pd.read_csv('labevents.csv', nrows=5000)

In [4]:
# extract existing subject_id from event_level_cleaned.csv
existing_subject_id = df_patients['subject_id'].unique()
existing_subject_id.shape

(203003,)

In [16]:
# changing format of df_sample
sepsis_labitems = {
    50813: "Lactate",
    50883: "Bilirubin, Direct",
    50885: "Bilirubin, Total",
    50886: "Blood Culture Hold",
    50889: "C-Reactive Protein",
    50912: "Creatinine",
    50915: "D-Dimer",
    51133: "Absolute Lymphocyte Count",
    51143: "Atypical Lymphocytes",
    51144: "Bands",
    51146: "Basophils",
    51199: "Eosinophil Count",
    51200: "Eosinophils",
    51237: "INR(PT)",
    51244: "Lymphocytes",
    51245: "Lymphocytes, Percent",
    51253: "Monocyte Count",
    51254: "Monocytes",
    51256: "Neutrophils",
    51265: "Platelet Count",
    51275: "PTT",
    51300: "WBC Count",
    51301: "White Blood Cells",
    51652: "High-Sensitivity CRP",
    51690: "Lymphocytes",
    51704: "Platelet Count",
    51755: "White Blood Cells",
    51756: "White Blood Cells",
    52069: "Absolute Basophil Count",
    52073: "Absolute Eosinophil Count",
    52074: "Absolute Monocyte Count",
    52075: "Absolute Neutrophil Count",
    52116: "Fibrinogen",
    52135: "Immature Granulocytes",
    52442: "Lactate",
    52769: "Absolute Lymphocyte Count",
    53132: "Absolute Lymphocyte Count",
    53133: "Absolute Neutrophil",
    53134: "Absolute Other WBC",
    53154: "Lactate",
}

WBC_COUNT_CODES = {
 51241: 'Leukocyte Alkaline Phosphatase',
 51300: 'WBC Count',
 51301: 'White Blood Cells',
 51486: 'Leukocytes',
 51516: 'WBC',
 51755: 'White Blood Cells',
 51756: 'White Blood Cells',
 52407: 'WBC',
 53134: 'Absolute Other WBC',
}






df_new_columns = ["subject_id", "hadm_id", "storetime"] + list(sepsis_labitems.values())
df = pd.DataFrame(columns=df_new_columns)
# first filter to relevant labitems
df_sample = df_sample[df_sample['itemid'].isin(sepsis_labitems.keys())]
#for each row, create a new column based on the itemid
for index, row in df_sample.iterrows():
    item_name = sepsis_labitems[row['itemid']]
    df.loc[index, 'subject_id'] = row['subject_id']
    df.loc[index, 'hadm_id'] = row['hadm_id']
    df.loc[index, 'storetime'] = row['storetime']
    df.loc[index, item_name] = row['valuenum']

# group by subject_id, hadm_id, storetime and take the sum of each column, keep nans as nans
df = df.groupby(['subject_id', 'storetime']).max().reset_index()
df

,subject_id,storetime,hadm_id,Lactate,"Bilirubin, Direct","Bilirubin, Total",Blood Culture Hold,C-Reactive Protein,Creatinine,D-Dimer,...,Absolute Monocyte Count,Absolute Neutrophil Count,Fibrinogen,Immature Granulocytes,Lactate,Absolute Lymphocyte Count,Absolute Lymphocyte Count,Absolute Neutrophil,Absolute Other WBC,Lactate
0,10000032,2180-03-23 15:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,2180-03-23 15:19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,2180-03-23 16:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,2180-03-23 16:40:00,NaN,NaN,NaN,1.6,NaN,NaN,0.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000032,2180-05-06 22:42:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,10000935,2187-08-27 08:37:00,26381316.0,NaN,NaN,0.8,NaN,NaN,0.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324,10000935,2187-08-27 08:54:00,26381316.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
325,10000935,2187-10-10 13:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326,10000935,2187-10-10 13:54:00,NaN,NaN,NaN,2.6,NaN,NaN,0.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# columns we ultimately want
df_new_columns = ["subject_id", "hadm_id", "storetime"] + list(sepsis_labitems.values())

# 1) filter once & map itemid → readable name
item_keys = set(sepsis_labitems)  # faster isin
df1 = df_sample.loc[df_sample['itemid'].isin(item_keys),
                    ['subject_id','hadm_id','storetime','itemid','valuenum']].copy()
df1['item_name'] = df1['itemid'].map(sepsis_labitems)

# 2) lab values wide per (subject_id, storetime) using vectorized groupby + unstack
wide = (df1.groupby(['subject_id','storetime','item_name'], sort=False)['valuenum']
          .max()
          .unstack('item_name'))           # same as pivot_table(..., aggfunc='max')
wide.columns.name = None

# 3) keep hadm_id: choose a non-null if any exist in the group (max ignores NaNs for numeric)
hadm = (df1.groupby(['subject_id','storetime'], sort=False)['hadm_id']
           .max())                         

# 4) assemble & order columns
df = (hadm.to_frame()
          .join(wide)
          .reset_index())

# ensure all expected lab columns exist and are ordered
df = df.reindex(columns=df_new_columns, fill_value=np.nan)
df


,subject_id,hadm_id,storetime,Lactate,"Bilirubin, Direct","Bilirubin, Total",Blood Culture Hold,C-Reactive Protein,Creatinine,D-Dimer,...,Absolute Monocyte Count,Absolute Neutrophil Count,Fibrinogen,Immature Granulocytes,Lactate,Absolute Lymphocyte Count,Absolute Lymphocyte Count,Absolute Neutrophil,Absolute Other WBC,Lactate
0,10000032,NaN,2180-03-23 15:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,NaN,2180-03-23 16:40:00,NaN,NaN,1.6,NaN,NaN,0.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,NaN,2180-03-23 15:19:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,NaN,2180-03-23 16:09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000032,NaN,2180-05-06 22:42:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,10000935,26381316.0,2187-08-27 08:54:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324,10000935,26381316.0,2187-08-27 08:37:00,NaN,NaN,0.8,NaN,NaN,0.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
325,10000935,NaN,2187-10-10 13:28:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326,10000935,NaN,2187-10-10 23:50:00,NaN,1.8,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
import pandas as pd
from typing import Dict, Union

def labevents_to_wide(
    input_file: str,
    output_file: str,
    sepsis_labitems: Dict[Union[int,str], str],
    chunksize: int = 1_000_000,
    parse_storetime: bool = True,
    output_format: str = "parquet",   # "parquet" | "csv"
):
    """
    Streams `input_file` (MIMIC labevents-like) and produces a wide table:
      index: (subject_id, storetime)
      columns: each lab in `sepsis_labitems` (values = max valuenum per group)
      plus: hadm_id = a non-null hadm_id if any exist in the group (uses max for speed)
    Only a compact running aggregate is kept in memory (unique groups x labs),
    so this scales far better than materializing all rows or writing an extra cleaned CSV.

    Parameters
    ----------
    input_file : path to large CSV of labevents
    output_file : path to final wide table (parquet/csv based on output_format)
    sepsis_labitems : {itemid: readable_name}
    chunksize : CSV chunk size
    parse_storetime : parse storetime as datetime (recommended for stable grouping)
    output_format : "parquet" (fast/compact) or "csv"
    """

    # ---- Read options: trim columns early and use lighter dtypes
    usecols = ['subject_id', 'hadm_id', 'itemid', 'storetime', 'valuenum']
    dtype = {
        'subject_id': 'int32',
        'itemid': 'int32',
        # hadm_id can be NaN -> use float to allow NaN, downcast later if you like
        'hadm_id': 'float32',
        'valuenum': 'float32',
    }
    parse_dates = ['storetime'] if parse_storetime else None

    item_keys = set(sepsis_labitems.keys())

    # Running (compact) aggregations
    labs_agg = None  # long form: [subject_id, storetime, item_name, valuenum_max]
    hadm_agg = None  # [subject_id, storetime, hadm_id_chosen]

    # Stream the file once
    for chunk in pd.read_csv(
        input_file,
        usecols=usecols,
        dtype=dtype,
        parse_dates=parse_dates,
        chunksize=chunksize
    ):
        # filter relevant itemids
        c = chunk[chunk['itemid'].isin(item_keys)].copy()
        if c.empty:
            continue

        # map itemid -> readable name (vectorized)
        c['item_name'] = c['itemid'].map(sepsis_labitems)

        # per-chunk aggregation (massively reduces rows)
        # labs: max value per (subject_id, storetime, item_name)
        labs_chunk = (
            c.groupby(['subject_id','storetime','item_name'], sort=False)['valuenum']
             .max()
             .rename('valuenum')
             .reset_index()
        )

        # hadm: choose a non-null if present within each (subject_id, storetime)
        # using max() over float column is simple & fast; it returns NaN if all NaN.
        hadm_chunk = (
            c.groupby(['subject_id','storetime'], sort=False)['hadm_id']
             .max()
             .reset_index()
        )

        # merge into running aggregates (still compact)
        if labs_agg is None:
            labs_agg = labs_chunk
        else:
            labs_agg = (
                pd.concat([labs_agg, labs_chunk], ignore_index=True)
                  .groupby(['subject_id','storetime','item_name'], sort=False)['valuenum']
                  .max()
                  .reset_index()
            )

        if hadm_agg is None:
            hadm_agg = hadm_chunk
        else:
            hadm_agg = (
                pd.concat([hadm_agg, hadm_chunk], ignore_index=True)
                  .groupby(['subject_id','storetime'], sort=False)['hadm_id']
                  .max()
                  .reset_index()
            )

    # If there were no matching rows at all
    if labs_agg is None:
        # Produce an empty frame with the expected columns
        df = pd.DataFrame(columns=["subject_id","hadm_id","storetime"] + list(sepsis_labitems.values()))
        if output_format == "parquet":
            df.to_parquet(output_file, index=False)
        else:
            df.to_csv(output_file, index=False)
        return

    # Pivot labs to wide once at the end
    wide = (
        labs_agg.pivot_table(
            index=['subject_id','storetime'],
            columns='item_name',
            values='valuenum',
            aggfunc='max'  # idempotent after our chunk-level max
        )
    )
    wide.columns.name = None
    wide = wide.reset_index()

    # Join hadm_id (already reduced to one value per (subject_id, storetime))
    df = hadm_agg.merge(wide, on=['subject_id','storetime'], how='right')

    # Order & ensure all expected lab columns exist
    lab_cols = list(sepsis_labitems.values())
    base_cols = ['subject_id','hadm_id','storetime']
    for col in lab_cols:
        if col not in df.columns:
            df[col] = pd.NA
    df = df[base_cols + lab_cols]

    # Persist
    if output_format == "parquet":
        df.to_parquet(output_file, index=False)
    else:
        df.to_csv(output_file, index=False)

    return df  # handy if you’re calling in a notebook

df = labevents_to_wide(
    input_file='labevents.csv',
    output_file='cleaned_labevents.csv',
    sepsis_labitems=sepsis_labitems,
    chunksize=1_000_000,
    parse_storetime=False,
    output_format='csv'
)

In [17]:
df_wbc = labevents_to_wide(
    input_file='labevents.csv',
    output_file='cleaned_labevents_wbc.csv',
    sepsis_labitems=WBC_COUNT_CODES,
    chunksize=1_000_000,
    parse_storetime=False,
    output_format='csv'
)

In [8]:
df_cleaned = pd.read_csv('cleaned_labevents.csv')
print(df_cleaned)

         subject_id     hadm_id            storetime  Lactate  \
0          10000032         NaN  2180-03-23 15:15:00      NaN   
1          10000032         NaN  2180-03-23 15:19:00      NaN   
2          10000032         NaN  2180-03-23 16:09:00      NaN   
3          10000032         NaN  2180-03-23 16:40:00      NaN   
4          10000032         NaN  2180-05-06 22:42:00      NaN   
...             ...         ...                  ...      ...   
9353644    19999987  23865744.0  2145-11-06 11:19:00      NaN   
9353645    19999987  23865744.0  2145-11-07 06:33:00      NaN   
9353646    19999987  23865744.0  2145-11-09 06:59:00      NaN   
9353647    19999987  23865744.0  2145-11-09 07:06:00      NaN   
9353648    19999987         NaN  2146-02-07 16:26:00      NaN   

         Bilirubin, Direct  Bilirubin, Total  Blood Culture Hold  \
0                      NaN               NaN                 NaN   
1                      NaN               NaN                 NaN   
2              

In [9]:
df_edstays = pd.read_csv('edstays.csv')
# we want to assign stay_id based on the time points in cleaned_labevents.csv
# if a lab event's storetime is between intime and outtime for a subject_id, assign that stay_id
# if there is no matching stay_id, drop the column
print(df_edstays)

        subject_id     hadm_id   stay_id               intime  \
0         10000032  22595853.0  33258284  2180-05-06 19:17:00   
1         10000032  22841357.0  38112554  2180-06-26 15:54:00   
2         10000032  25742920.0  35968195  2180-08-05 20:58:00   
3         10000032  29079034.0  32952584  2180-07-22 16:24:00   
4         10000032  29079034.0  39399961  2180-07-23 05:54:00   
...            ...         ...       ...                  ...   
425082    19999784  26194817.0  35692999  2119-06-18 14:21:00   
425083    19999828  25744818.0  32917002  2149-01-08 09:11:00   
425084    19999828  29734428.0  30712109  2147-07-17 17:18:00   
425085    19999914         NaN  32002659  2158-12-24 11:41:00   
425086    19999987  23865745.0  34731548  2145-11-02 19:28:00   

                    outtime gender                    race arrival_transport  \
0       2180-05-06 23:30:00      F                   WHITE         AMBULANCE   
1       2180-06-26 21:31:00      F                   WHITE 

In [10]:
import pandas as pd

# 1) Parse datetimes
df_cleaned['storetime'] = pd.to_datetime(df_cleaned['storetime'], errors='coerce')
df_edstays['intime']    = pd.to_datetime(df_edstays['intime'],    errors='coerce')
df_edstays['outtime']   = pd.to_datetime(df_edstays['outtime'],   errors='coerce')

# 2) Prep working copies and handle missing
labs = df_cleaned.dropna(subset=['storetime']).copy()
ed   = df_edstays.dropna(subset=['intime']).copy()
ed['outtime'] = ed['outtime'].fillna(pd.Timestamp.max)

# (strongly recommended) match dtypes for the 'by' key
labs['subject_id'] = pd.to_numeric(labs['subject_id'], errors='coerce')
ed['subject_id']   = pd.to_numeric(ed['subject_id'],   errors='coerce')

# 3) SORT by the *on* keys GLOBALLY (this is what merge_asof checks)
labs = labs.sort_values('storetime', kind='mergesort')
ed   = ed.sort_values('intime',    kind='mergesort')

# (Optional sanity checks)
# assert labs['storetime'].is_monotonic_increasing
# assert ed['intime'].is_monotonic_increasing

# 4) As-of merge within subject_id
merged = pd.merge_asof(
    labs,
    ed[['subject_id','stay_id','intime','outtime']],
    left_on='storetime',
    right_on='intime',
    by='subject_id',
    direction='backward',
    allow_exact_matches=True
)

# 5) Keep only events inside the stay window
mask = merged['storetime'] <= merged['outtime']
merged.loc[~mask, 'stay_id'] = pd.NA

# 6) Attach stay_id back to the original + drop unmatched rows
df_cleaned = df_cleaned.merge(
    merged[['subject_id','storetime','stay_id']],
    on=['subject_id','storetime'],
    how='left'
)

df_cleaned = df_cleaned.dropna(subset=['stay_id']).copy()  # drop lab rows with no matching stay

df_cleaned


,subject_id,hadm_id,storetime,Lactate,"Bilirubin, Direct","Bilirubin, Total",Blood Culture Hold,C-Reactive Protein,Creatinine,D-Dimer,...,Absolute Neutrophil Count,Fibrinogen,Immature Granulocytes,Lactate.1,Absolute Lymphocyte Count.1,Absolute Lymphocyte Count.2,Absolute Neutrophil,Absolute Other WBC,Lactate.2,stay_id
4,10000032,NaN,2180-05-06 22:42:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33258284.0
5,10000032,NaN,2180-05-06 23:13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33258284.0
6,10000032,NaN,2180-05-06 23:14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33258284.0
7,10000032,NaN,2180-05-06 23:16:00,NaN,NaN,1.6,NaN,NaN,0.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33258284.0
15,10000032,NaN,2180-06-26 16:40:00,1.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.7,NaN,NaN,NaN,NaN,1.7,38112554.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9353571,19999828,NaN,2149-01-08 16:34:00,NaN,NaN,NaN,NaN,NaN,0.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32917002.0
9353630,19999987,NaN,2145-11-02 19:51:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34731548.0
9353633,19999987,NaN,2145-11-02 19:54:00,1.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.7,NaN,NaN,NaN,NaN,1.7,34731548.0
9353634,19999987,NaN,2145-11-02 20:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34731548.0


In [11]:
# reorder columns
df_cleaned = df_cleaned[['subject_id', 'hadm_id', 'stay_id', 'storetime'] + list(sepsis_labitems.values())]
df_cleaned

,subject_id,hadm_id,stay_id,storetime,Lactate,"Bilirubin, Direct","Bilirubin, Total",Blood Culture Hold,C-Reactive Protein,Creatinine,...,Absolute Monocyte Count,Absolute Neutrophil Count,Fibrinogen,Immature Granulocytes,Lactate,Absolute Lymphocyte Count,Absolute Lymphocyte Count,Absolute Neutrophil,Absolute Other WBC,Lactate
4,10000032,NaN,33258284.0,2180-05-06 22:42:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10000032,NaN,33258284.0,2180-05-06 23:13:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10000032,NaN,33258284.0,2180-05-06 23:14:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10000032,NaN,33258284.0,2180-05-06 23:16:00,NaN,NaN,1.6,NaN,NaN,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,10000032,NaN,38112554.0,2180-06-26 16:40:00,1.7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.7,NaN,NaN,NaN,NaN,1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9353571,19999828,NaN,32917002.0,2149-01-08 16:34:00,NaN,NaN,NaN,NaN,NaN,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9353630,19999987,NaN,34731548.0,2145-11-02 19:51:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9353633,19999987,NaN,34731548.0,2145-11-02 19:54:00,1.7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.7,NaN,NaN,NaN,NaN,1.7
9353634,19999987,NaN,34731548.0,2145-11-02 20:05:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_cleaned.to_csv("cleaned_labevents.csv", index=False)

In [13]:
# made stay_id into int
import pandas as pd
df_cleaned = pd.read_csv('cleaned_labevents.csv')
df_cleaned['stay_id'] = df_cleaned['stay_id'].astype('int32')
df_cleaned['hadm_id'] = df_cleaned['hadm_id'].astype('Int32')  # if you want hadm_id as nullable Int
df_cleaned
df_cleaned.to_csv("cleaned_labevents.csv", index=False)

In [33]:
df_wbc = pd.read_csv('cleaned_labevents_wbc.csv')
df_wbc

,subject_id,hadm_id,storetime,Leukocyte Alkaline Phosphatase,WBC Count,White Blood Cells,Leukocytes,WBC,White Blood Cells.1,White Blood Cells.2,WBC.1,Absolute Other WBC
0,10000032,NaN,2180-03-23 15:19:00,NaN,NaN,3.0,NaN,NaN,3.0,3.0,NaN,NaN
1,10000032,NaN,2180-05-06 22:42:00,NaN,NaN,5.0,NaN,NaN,5.0,5.0,NaN,NaN
2,10000032,22595852.0,2180-05-07 06:22:00,NaN,NaN,4.2,NaN,NaN,4.2,4.2,NaN,NaN
3,10000032,NaN,2180-06-22 14:50:00,NaN,NaN,5.1,NaN,NaN,5.1,5.1,NaN,NaN
4,10000032,NaN,2180-06-26 16:50:00,NaN,NaN,6.6,NaN,NaN,6.6,6.6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3539243,19999987,23865744.0,2145-11-04 11:30:00,NaN,NaN,NaN,NaN,136.0,NaN,NaN,136.0,NaN
3539244,19999987,23865744.0,2145-11-05 06:34:00,NaN,NaN,10.0,NaN,NaN,10.0,10.0,NaN,NaN
3539245,19999987,23865744.0,2145-11-06 10:37:00,NaN,NaN,5.9,NaN,NaN,5.9,5.9,NaN,NaN
3539246,19999987,23865744.0,2145-11-07 06:33:00,NaN,NaN,5.0,NaN,NaN,5.0,5.0,NaN,NaN


In [34]:
# count nans in each column
df_wbc.isna().sum()

subject_id                              0
hadm_id                           1660386
storetime                               0
Leukocyte Alkaline Phosphatase    3539200
WBC Count                         3513103
White Blood Cells                  375352
Leukocytes                        3539247
WBC                               3188451
White Blood Cells.1                375352
White Blood Cells.2                375352
WBC.1                             3188451
Absolute Other WBC                3539248
dtype: int64

In [38]:
# see where WBC are nans
df_wbc_empty = df_wbc[df_wbc['White Blood Cells'].isna()]
df_wbc_nonempty = df_wbc[df_wbc['White Blood Cells'].notna()]
df_wbc_empty = df_wbc_empty[df_wbc_empty["WBC"].notna()]
df_wbc_empty

,subject_id,hadm_id,storetime,Leukocyte Alkaline Phosphatase,WBC Count,White Blood Cells,Leukocytes,WBC,White Blood Cells.1,White Blood Cells.2,WBC.1,Absolute Other WBC
5,10000032,NaN,2180-06-26 19:02:00,NaN,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN
6,10000032,22841356.0,2180-06-27 00:38:00,NaN,NaN,NaN,NaN,13.0,NaN,NaN,13.0,NaN
17,10000084,NaN,2160-11-21 01:17:00,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
23,10000084,NaN,2160-12-28 01:21:00,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
26,10000117,NaN,2174-08-14 16:48:00,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3539188,19999784,NaN,2122-05-05 08:34:00,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
3539208,19999828,29734428.0,2147-08-01 22:35:00,NaN,NaN,NaN,NaN,13.0,NaN,NaN,13.0,NaN
3539227,19999840,26071774.0,2164-07-25 06:41:00,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN
3539239,19999987,NaN,2145-11-02 19:57:00,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN


In [39]:
# only keep columns subject_id, hadm_id, storetime, White Blood Cells
df_wbc_simple = df_wbc_nonempty.copy().loc[:, ['subject_id', 'hadm_id', 'storetime', 'White Blood Cells']]
df_wbc_empty_simple = df_wbc_empty.copy().loc[:, ['subject_id', 'hadm_id', 'storetime', 'WBC']]
#rename WBC to White Blood Cells
df_wbc_empty_simple = df_wbc_empty_simple.rename(columns={"WBC": "White Blood Cells"})
df_wbc_simple_combined = pd.concat([df_wbc_simple, df_wbc_empty_simple], ignore_index=True).sort_values(by=["storetime"])
df_wbc_simple_combined

,subject_id,hadm_id,storetime,White Blood Cells
2180897,16904137,NaN,2105-01-19 12:26:00,7.2
2180898,16904137,NaN,2105-02-02 11:38:00,8.9
2180899,16904137,NaN,2105-04-06 10:46:00,7.7
2180900,16904137,NaN,2105-10-04 17:56:00,9.6
2180901,16904137,21081216.0,2105-10-05 02:33:00,11.5
...,...,...,...,...
615436,11973788,NaN,2212-11-14 14:11:00,5.2
1666993,15273135,NaN,2212-12-05 19:25:00,7.2
2078674,16573705,NaN,2212-12-19 21:09:00,4.6
3394063,16573705,NaN,2212-12-19 22:24:00,24.0


In [40]:
import pandas as pd

df_edstays = pd.read_csv('edstays.csv')

# 1) Parse datetimes
df_wbc_simple_combined['storetime'] = pd.to_datetime(df_wbc_simple_combined['storetime'], errors='coerce')
df_edstays['intime']    = pd.to_datetime(df_edstays['intime'],    errors='coerce')
df_edstays['outtime']   = pd.to_datetime(df_edstays['outtime'],   errors='coerce')

# 2) Prep working copies and handle missing
wbcs = df_wbc_simple_combined.dropna(subset=['storetime']).copy()
ed   = df_edstays.dropna(subset=['intime']).copy()
ed['outtime'] = ed['outtime'].fillna(pd.Timestamp.max)

# (strongly recommended) match dtypes for the 'by' key
wbcs['subject_id'] = pd.to_numeric(wbcs['subject_id'], errors='coerce')
ed['subject_id']   = pd.to_numeric(ed['subject_id'],   errors='coerce')

# 3) SORT by the *on* keys GLOBALLY (this is what merge_asof checks)
wbcs = wbcs.sort_values('storetime', kind='mergesort')
ed   = ed.sort_values('intime',    kind='mergesort')

# (Optional sanity checks)
# assert labs['storetime'].is_monotonic_increasing
# assert ed['intime'].is_monotonic_increasing

# 4) As-of merge within subject_id
merged = pd.merge_asof(
    wbcs,
    ed[['subject_id','stay_id','intime','outtime']],
    left_on='storetime',
    right_on='intime',
    by='subject_id',
    direction='backward',
    allow_exact_matches=True
)

# 5) Keep only events inside the stay window
mask = merged['storetime'] <= merged['outtime']
merged.loc[~mask, 'stay_id'] = np.nan

# 6) Attach stay_id back to the original + drop unmatched rows
df_wbc_simple_combined = df_wbc_simple_combined.merge(
    merged[['subject_id','storetime','stay_id']],
    on=['subject_id','storetime'],
    how='left'
)

df_wbc_simple_combined = df_wbc_simple_combined.dropna(subset=['stay_id']).copy()  # drop lab rows with no matching stay


#reorder columns so stay_id is after hadm_id
df_wbc_simple_combined = df_wbc_simple_combined[['subject_id', 'hadm_id', 'stay_id', 'storetime', 'White Blood Cells']]
# make stay_id int
df_wbc_simple_combined['stay_id'] = df_wbc_simple_combined['stay_id'].astype('int32')
# make hadm_id Int32
df_wbc_simple_combined['hadm_id'] = df_wbc_simple_combined['hadm_id'].astype('Int32')
df_wbc_simple_combined


,subject_id,hadm_id,stay_id,storetime,White Blood Cells
623,15350437,<NA>,39042378,2110-01-11 05:18:00,10.6
626,13370527,<NA>,35220003,2110-01-11 11:02:00,9.8
629,14695442,<NA>,34789981,2110-01-11 13:45:00,7.4
632,17195991,<NA>,38649090,2110-01-11 22:24:00,11.4
633,17195991,<NA>,38649090,2110-01-11 22:33:00,41.0
...,...,...,...,...,...
3512943,11973788,23238116,37355521,2212-01-27 22:42:00,5.0
3512944,11973788,<NA>,37355521,2212-01-28 09:41:00,5.0
3512978,11973788,<NA>,33494247,2212-04-06 01:07:00,4.8
3512979,11973788,<NA>,33494247,2212-04-06 08:23:00,4.2


In [41]:
df_wbc_simple_combined.to_csv("cleaned_labevents_wbc.csv", index=False)